In [2]:
import json_lines
import pandas as pd
import numpy as np
import datetime
import itertools

In [3]:
# convert raw .jl file to dataframe
tmp_df = pd.DataFrame()
for fn in ['raw_weather-data_2014.jl', 'raw_weather-data_2015.jl']:
    with open(fn, 'rb') as f:
        tmp_df = tmp_df.append(pd.DataFrame(json_lines.reader(f)), ignore_index=True)

display(tmp_df)
# fix Date, Time column
tmp_df['Time'] = [int(t[:2]) for t in tmp_df['Time']]
tmp_df['Date'] = pd.to_datetime(tmp_df['Date'])
tmp_df = tmp_df.drop_duplicates(subset=['Date', 'Time'], keep='first').reset_index(drop=True)
tmp_df.set_index(['Date', 'Time'], inplace=True)

# remove unit symbols
tmp_df['Temperature'] = [int(t[:-2]) for t in tmp_df['Temperature']]
tmp_df['Relative Temperature'] = [int(t[:-2]) for t in tmp_df['Relative Temperature']]
tmp_df['Wind'] = [float(t[:-4]) if t else None for t in tmp_df['Wind']]
tmp_df['Rel. Humidity'] = [int(t[:-1]) for t in tmp_df['Rel. Humidity']]
tmp_df['Pressure'] = [float(t[:-2]) for t in tmp_df['Pressure']]

# fill missing value in wind column
tmp_df['Wind'] = tmp_df['Wind'].fillna(0)

# ignore pressure
tmp_df.drop(['Pressure'], axis=1, inplace=True)

# ignore description
tmp_df.drop(['Description'], axis=1, inplace=True)

# create complete datatime range
drng = pd.date_range(start='2014-01-01', end='2015-12-31')
hrng = range(24)
midx = pd.MultiIndex.from_product([drng, hrng], names=["Date", "Time"])
df = pd.DataFrame(index=midx, columns=tmp_df.columns)
df.loc[tmp_df.index] = tmp_df

# fill missing records
df = df.fillna(method='ffill', axis=0)

display(df)
df.info()

,Date,Time,Temperature,Relative Temperature,Wind,Rel. Humidity,Pressure,Description
0,2014-01-01,00:52,1°C,1°C,None,48%,1027.0mb,Clear below 3700m 6-Hour Maximum Temperature...
1,2014-01-01,01:52,-1°C,-5°C,13 Km/h,59%,1027.0mb,Scattered Clouds at 1500m
2,2014-01-01,02:52,-1°C,-3°C,6 Km/h,59%,1028.0mb,Few Clouds at 1700m
3,2014-01-01,03:52,1°C,1°C,None,48%,1028.0mb,"Few Clouds at 4900m, Broken Clouds at 6100m ..."
4,2014-01-01,04:52,-1°C,-1°C,None,64%,1029.0mb,"Few Clouds at 4900m, Cloudy at 6100m"
...,...,...,...,...,...,...,...,...
21987,2015-12-31,19:52,9°C,6°C,22 Km/h,66%,1021.0mb,"Broken Clouds at 1400m, Cloudy at 1700m"
21988,2015-12-31,20:52,9°C,6°C,17 Km/h,62%,1021.0mb,Cloudy at 1300m
21989,2015-12-31,21:52,8°C,4°C,28 Km/h,62%,1021.0mb,Cloudy at 1300m 3-Hour Pressure Decreasing: ...
21990,2015-12-31,22:52,8°C,5°C,15 Km/h,62%,1021.0mb,Cloudy at 1300m


Temperature  Relative Temperature  Wind  Rel. Humidity
Date       Time                                                        
2014-01-01 0             1.0                   1.0   0.0           48.0
           1            -1.0                  -5.0  13.0           59.0
           2            -1.0                  -3.0   6.0           59.0
           3             1.0                   1.0   0.0           48.0
           4            -1.0                  -1.0   0.0           64.0
...                      ...                   ...   ...            ...
2015-12-31 19            9.0                   6.0  22.0           66.0
           20            9.0                   6.0  17.0           62.0
           21            8.0                   4.0  28.0           62.0
           22            8.0                   5.0  15.0           62.0
           23            7.0                   4.0  19.0           61.0

[17520 rows x 4 columns]

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 17520 entries, (2014-01-01 00:00:00, 0) to (2015-12-31 00:00:00, 23)
Data columns (total 4 columns):
Temperature             17520 non-null float64
Relative Temperature    17520 non-null float64
Wind                    17520 non-null float64
Rel. Humidity           17520 non-null float64
dtypes: float64(4)
memory usage: 1.2 MB


In [86]:
df.to_csv('weather-data.csv')

In [87]:
# normalizing metric using min-max normalizer
df = (df-df.min())/(df.max()-df.min())
display(df)

Temperature  Relative Temperature      Wind  Rel. Humidity
Date       Time                                                            
2014-01-01 0        0.241935              0.367647  0.000000       0.440860
           1        0.209677              0.279412  0.213115       0.559140
           2        0.209677              0.308824  0.098361       0.559140
           3        0.241935              0.367647  0.000000       0.440860
           4        0.209677              0.338235  0.000000       0.612903
...                      ...                   ...       ...            ...
2015-12-31 19       0.370968              0.441176  0.360656       0.634409
           20       0.370968              0.441176  0.278689       0.591398
           21       0.354839              0.411765  0.459016       0.591398
           22       0.354839              0.426471  0.245902       0.591398
           23       0.338710              0.411765  0.311475       0.580645

[17520 rows x 4 columns]

In [88]:
df.to_csv('weather-data-normalized.csv')